In [1]:
import cv2
import numpy as np

# 将文本转换为二进制
def text_to_binary(text):
    # print("binary text: ", ''.join(format(ord(char), '08b') for char in text))
    return ''.join(format(ord(char), '08b') for char in text) + '1111111111111110'  # 结束标志

# 将二进制转换为文本
def binary_to_text(binary):
    # print("binary: ", binary)
    # print("type: ", type(binary))
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    # print("text: ", ''.join(chr(int(char, 2)) for char in chars if int(char, 2) != 0))
    return ''.join(chr(int(char, 2)) for char in chars if int(char, 2) != 0)

# 隐藏文本函数，采用灰度值修改技术
# def hide_text(cover_path, text, stego_path):
#     cover = cv2.imread(cover_path, cv2.IMREAD_GRAYSCALE)  # 以灰度图像读取
#     binary_text = text_to_binary(text)
#     data_index = 0
#     data_length = len(binary_text)
#     print("inner data_length: ", data_length)
#     print("inner binary_text: ", binary_text)
#     # print 10 first pixels
#     for row in cover:
#         print(row[:10])
#         break
#     print("cover shape: ", cover.shape)
#     for row in cover:
#         for pixel in row:
#             if data_index < data_length:
#                 # print("data index: ", data_index)
#                 # print("all text: ", binary_text)        
#                 # print("text: ", binary_text[data_index])        
#                 # # 只操作灰度值，修改最末一位
#                 # if pixel % 2 == 0:  # 如果是偶数，修改为表示0的灰度值
#                 #     pixel = pixel if binary_text[data_index] == '1' else pixel
#                 # else:  # 如果是奇数，修改为表示1的灰度值
#                 #     pixel = pixel + 1 if binary_text[data_index] == '1' else pixel
#                 pixel_value = pixel
#                 original_pixel = pixel_value
#                 pixel_value = pixel_value - pixel_value % 2  # 保证最末一位为0
#                 inner_pixel = pixel_value
#                 if binary_text[data_index] == '1':
#                     pixel_value += 1
#                 elif binary_text[data_index] == '0':
#                     pass
#                 else:
#                     print("Error: Invalid binary data")
#                     return
#                 # print(f"original pixel: {original_pixel}, now pixel: {pixel_value}, inner_pixel: {inner_pixel}, binary_text[data_index]: {binary_text[data_index]}")
#                 data_index += 1
#                 cover[row][pixel] = pixel_value
                
#             if data_index >= data_length:
#                 break
#     # print 10 first pixels
#     for row in cover:
#         print(row[:10])
#         break
#     cv2.imwrite(stego_path, cover)
#     print("Text hidden successfully!")

def hide_text(cover_path, text, stego_path):
    cover = cv2.imread(cover_path, cv2.IMREAD_GRAYSCALE)  # 以灰度图像读取
    binary_text = text_to_binary(text)
    data_index = 0
    data_length = len(binary_text)

    rows, cols = cover.shape
    for i in range(rows):
        for j in range(cols):
            if data_index < data_length:
                pixel_value = cover[i, j]
                pixel_value = pixel_value - pixel_value % 2
                if binary_text[data_index] == '1':
                    pixel_value += 1
                elif binary_text[data_index] == '0':
                    pass
                else:
                    print("Error: Invalid binary data")
                    return
                # print(f"original pixel: {original_pixel}, now pixel: {pixel_value}, inner_pixel: {inner_pixel}, binary_text[data_index]: {binary_text[data_index]}")
                cover[i, j] = pixel_value
                data_index += 1
            if data_index >= data_length:
                break
        if data_index >= data_length:
            break

    cv2.imwrite(stego_path, cover)
    print("Text hidden successfully!")

# 提取文本函数，提取灰度值中嵌入的数据
def extract_text(stego_path):
    stego = cv2.imread(stego_path, cv2.IMREAD_GRAYSCALE)
    binary_text = ''
    
    for row in stego:
        for pixel in row:
            # print("pixel: ", pixel)
            binary_text += str(pixel % 2)  # 获取灰度值的最低位
            if binary_text[-16:] == '1111111111111110':  # 检测到结束标志
                return binary_to_text(binary_text[:-16])
    return "Message not found"

# 示例用法
hide_text('./dataset/cover_gray.jpg', 'Hi! This is our GLM', './results/stego_glm.png')
print(extract_text('./results/stego_glm.png'))


Text hidden successfully!
Hi! This is our GLM
